In [1]:
import pandas as pd
import psycopg2
from psycopg2 import sql
import config
import os
import glob
from sqlalchemy import create_engine
import sqlalchemy

In [2]:
dbname = config.dbname
user = config.user
password = config.password
host = config.host
port = config.port


# Database credentials
db_credentials = {
    'dbname': config.dbname,
    'user': config.user,
    'password': config.password,
    'host': config.host,
    'port': config.port
}

In [18]:
directory =r"D:/eve_database/player_count/daily_player_count.csv"

In [10]:
# Function to create table and insert data
def create_table(dbname, user, password, host, port, table_name):
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    cur = conn.cursor()
    print("Check 2")
    # Create table if it doesn't exist
    cur.execute(sql.SQL("""
        CREATE TABLE IF NOT EXISTS {}
        (
            date DATE NOT NULL,
            player_count INT NOT NULL
        );
        SELECT create_hypertable('player_count', 'date');
    """).format(sql.Identifier(table_name)))
    # Commit changes and close connection
    conn.commit()
    cur.close()
    conn.close()

In [11]:
create_table(dbname, user, password, host, port, "player_count")

Check 2


In [26]:
import os
import pandas as pd
from sqlalchemy import create_engine

def load_csv_to_timescaledb(table_name, csv_folder_path, db_credentials):
    # Create a connection to the PostgreSQL database
    engine = create_engine(f'postgresql+psycopg2://{db_credentials["user"]}:{db_credentials["password"]}@{db_credentials["host"]}:{db_credentials["port"]}/{db_credentials["dbname"]}')

    file_path = os.path.join(csv_folder_path)
    print(f"Loading {file_path} into database...")

    # Read CSV file into DataFrame
    df = pd.read_csv(file_path)

    # Insert DataFrame into database table
    df.to_sql(table_name, engine, if_exists='append', index=False, method='multi')

    # Close the database connection
    engine.dispose()



# Run the function
load_csv_to_timescaledb("player_count", directory, db_credentials)


Loading D:/eve_database/player_count/daily_player_count.csv into database...
